# Tabular Playground August - Regression using GLM 
## Table of Contents
* [Import Data / First Glance](#1)
* [EDA](#2)
* [Fit Linear Model](#3)
* [Evaluate Model on Training Data](#4)
* [Build Submission](#5)

In [ ]:
# packages

# standard
import numpy as np
import pandas as pd
import time

# plots
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# statistics tools
import scipy.stats as stats
from sklearn.metrics import mean_absolute_error, mean_squared_error

# machine learning tools
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

<a id='1'></a>
# Import Data / First Glance

In [ ]:
# load data / preview
df_train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
df_sub = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
n_train = df_train.shape[0]
df_train.shape

In [ ]:
n_test = df_test.shape[0]
df_test.shape

<a id='2'></a>
# EDA

In [ ]:
# plot target
plt.figure(figsize=(12,4))
df_train.loss.plot(kind='hist', bins=25)
plt.title('Target - Histogram')
plt.grid()
plt.show()

In [ ]:
# categorical plot of target
plt.figure(figsize=(12,4))
df_train.loss.value_counts().sort_index().plot(kind='bar')
plt.title('Target - Discrete Distribution')
plt.grid()
plt.show()

In [ ]:
# features
features = df_test.columns
features = features.drop('id')
features = features.to_list()

In [ ]:
# evaluate correlations with target
corr_stats = pd.DataFrame(data=features, columns=['feature'])
corr_stats['corr_pearson'] = np.zeros(len(features))
corr_stats['corr_spearman'] = np.zeros(len(features))

i = 0
for f in features:
    c = df_train[f].corr(df_train.loss, method='pearson')
    c = np.round(c,4)
    corr_stats.loc[i,'corr_pearson'] = c
    c = df_train[f].corr(df_train.loss, method='spearman')
    c = np.round(c,4)    
    corr_stats.loc[i,'corr_spearman'] = c
    i=i+1

In [ ]:
# show top correlations (positive)
corr_stats.nlargest(10, columns='corr_pearson')

In [ ]:
# scatter plot with regression line
df_temp = pd.DataFrame({'Feature 13': df_train.f13, 
                        'Target': df_train.loss})
sns.jointplot(data=df_temp, x='Feature 13', y='Target',
              kind='reg',
              joint_kws={'line_kws':{'color':'magenta'}, 
                         'scatter_kws': {'alpha': 0.05}})
plt.show()

In [ ]:
# show top correlations (negative)
corr_stats.nsmallest(10, columns='corr_pearson')

In [ ]:
# scatter plot with regression line
df_temp = pd.DataFrame({'Feature 25': df_train.f25, 
                        'Target': df_train.loss})
sns.jointplot(data=df_temp, x='Feature 25', y='Target',
              kind='reg',
              joint_kws={'line_kws':{'color':'magenta'}, 
                         'scatter_kws': {'alpha': 0.05}})
plt.show()

#### Ok, we see pretty weak correlations with the target! We cannot expect a really good model here...

<a id='3'></a>
# Fit Linear Model

In [ ]:
# define target
target = 'loss'

In [ ]:
# start H2O
h2o.init(max_mem_size='12G', nthreads=4) # Use maximum of 12 GB RAM and 4 cores

In [ ]:
# upload data in H2O environment
t1 = time.time()
train_hex = h2o.H2OFrame(df_train)
test_hex = h2o.H2OFrame(df_test)
t2 = time.time()
print('Elapsed time [s]:', np.round(t2-t1,4))

In [ ]:
# define GLM
glm_model = H2OGeneralizedLinearEstimator(family = 'gaussian',
                                          nfolds = 5,
                                          alpha = 0, 
                                          # 0: Ridge (L2), 1: LASSO (L1)
                                          lambda_search = True,
                                          score_each_iteration = True,                                          
                                          seed=12345)

In [ ]:
# train model
t1 = time.time()
glm_model.train(features, target, training_frame = train_hex)
t2 = time.time()
print('Elapsed time [s]:', np.round(t2-t1,4))

In [ ]:
# show model details
glm_model

<a id='4'></a>
# Evaluate Model on Training Data

In [ ]:
# variable importance
glm_model.varimp_plot(25)

In [ ]:
# predict on training data
pred_train = glm_model.predict(train_hex)
y_train_act = train_hex.as_data_frame()[target].values # actuals
y_train_pred = pred_train.as_data_frame().predict.values # predictions

In [ ]:
# plot distribution of predictions
plt.hist(y_train_pred, bins=100)
plt.title('Predictions on Training Data')
plt.grid()
plt.show()

In [ ]:
# plot predictions vs actual (training)
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax.scatter(x=y_train_act, y=y_train_pred, alpha=0.1)
ax.plot([0,45],[0,45], color='green')
ax.set_aspect(1)
plt.grid()
plt.title('Prediction vs Actual - Training Data')
plt.xlabel('Actual')
plt.ylabel('Prediction')
plt.show()

#### Well, it seems that there is not really much signal in the data...

In [ ]:
# scatter plot with regression line
df_temp = pd.DataFrame({'Actual': y_train_act, 
                        'Prediction': y_train_pred})
sns.jointplot(data=df_temp, x='Actual', y='Prediction',
              kind='reg',
              joint_kws={'line_kws':{'color':'magenta'}, 
                         'scatter_kws': {'alpha': 0.1}})
plt.show()

In [ ]:
# yet another viz
sns.jointplot(data=df_temp, x='Actual', y='Prediction',
              kind='kde')
plt.show()

In [ ]:
# correlations
print('Correlations - Training Data')
print('Correlation Pearson:', stats.pearsonr(y_train_act, y_train_pred))
print('Correlation Spearman:', stats.spearmanr(y_train_act, y_train_pred))

In [ ]:
# metrics on training data
print('MAE (train):', np.round(mean_absolute_error(y_train_act, y_train_pred),4))
print('RMSE(train):', np.round(np.sqrt(mean_squared_error(y_train_act, y_train_pred)),4))

In [ ]:
# trivial benchmark for comparison: use simple mean of target
m = y_train_act.mean()
RMSE_train_trivial = np.sqrt(np.dot(y_train_act-m,y_train_act-m)/n_train)
print('RMSE(train,trivial model):', np.round(RMSE_train_trivial,4))

<a id='5'></a>
# Build Submission

In [ ]:
# predict on test data
pred_test = glm_model.predict(test_hex)
y_test_pred = pred_test.as_data_frame().predict.values # predictions

In [ ]:
# fill submission
df_sub.loss = y_test_pred
df_sub

In [ ]:
# and save result
df_sub.to_csv('submission.csv', index=None)